### קוד מבוא

In [955]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [956]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [957]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [958]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [959]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [960]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [961]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [962]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [963]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


### ביצוע

#### העלת שכבות

In [964]:
taz=up_load_shp(r'W:\Model Versions\4.0\TAZ\TAZ_V4_221212_Published.shp')

In [965]:
taz=taz.query('main_secto=="Arab" | main_secto=="arabs_behined_seperation_wall" ').set_index('Taz_num')

In [966]:
taz_point=make_point(taz)

#### יצירת אוכלוסיה לפי קיבולת

תוספת קיבולת אפשרית

In [967]:
path=r'K:\projections_team\תכניות אסטרטגיות לתחבורה ירושלים\תחזיות 2050\קיבולת_ערבי_מחוז_ירושלים\kibolet_with_behind_wall_230709.shp'

In [968]:
kibolet_with_behind_wall=up_load_shp(path).set_index('Taz_num')

יח"ד דיור קיימות

In [969]:
folder_path=r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\output'

In [970]:
file_name='2020_jtmt_forcast_full_230710'

In [971]:
taz_kayim=up_load_df(folder_path,file_name).set_index('Taz_num')

ג"מ

In [972]:
path=r'\\FILE-SRV\Jtmt\projections_team\תכניות אסטרטגיות לתחבורה ירושלים\תחזיות 2050\קבלת מידע\גודל משק בית\hh_size_2050.shp'

In [973]:
hh_size_2050=up_load_shp(path)

In [974]:
hh_size_2050=taz_point.sjoin(hh_size_2050,how='left')[['2050_hh_si']]

In [975]:
taz['hh_size']=hh_size_2050['2050_hh_si']

הכנה ללופ : מצב קיים הופך למצב נוכחי

In [976]:
taz['current_pop']=taz_kayim['pop']

In [977]:
taz['pop_2020']=taz_kayim['pop']

In [978]:
taz['current_hh']=taz_kayim['aprt_20']

In [979]:
taz['hh_2020']=taz_kayim['aprt_20']

להוציא את האזורים שמחוץ לירושלים

In [980]:
taz_out_of_jeru=taz.loc[taz.index.isin([7651.0, 7652.0])]

In [981]:
taz=taz.loc[~taz.index.isin([7651.0, 7652.0])]

עכשיו חישוב של כמות אוכלוסיה כל חומש

In [982]:
taz['pop_kibolet']=taz['hh_size']*kibolet_with_behind_wall['kibolet']

הכנה ללופ : הלאה של אוכלסייה פלסטינאית

In [983]:
folder_path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2020\מרכיבי_תחזית'

file_name='taz_demo_pls_2020_and_pre_growth_till_2050'

taz_demo_pls=up_load_df(folder_path,file_name)

taz_demo_pls['current_pop']=taz_demo_pls['pop_2020']

#### לופ לייצר אנשים כל חומש

In [984]:
col_age=['Taz_num','pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

חישוב מחוץ לירושלים

In [985]:
taz_out_of_jeru_point=make_point(taz_out_of_jeru)

In [986]:
year=['2025','2030','2035','2040','2045','2050']

for y in year:
    # לחבר למצב הנוכחי את הגידול והתפלגות שהוא צריך לקבל
    path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\עבודת_דמוגרף\SHP\pre_demo_growth_dislov_{}.shp'.format(y)

    demo=up_load_shp(path)  

    demand=taz_out_of_jeru_point.sjoin(demo,how='left')

    taz_age=demand.reset_index()[col_age]
    
    taz_out_of_jeru_point['add_current_pop']=taz_out_of_jeru_point['current_pop']*demand['precent_ad']-taz_out_of_jeru_point['current_pop']
    
    taz_out_of_jeru_point['current_pop']=taz_out_of_jeru_point['current_pop']+taz_out_of_jeru_point['add_current_pop']
    
    taz_out_of_jeru_point['current_hh']=taz_out_of_jeru_point['current_hh']+taz_out_of_jeru_point['add_current_pop']/taz_out_of_jeru_point['hh_size']
    
    taz_out_of_jeru_point['hh_{}'.format(y)]=taz_out_of_jeru_point['current_hh']
    
    taz_out_of_jeru_point['pop_{}'.format(y)]=taz_out_of_jeru_point['current_pop']

    
    col=['Taz_num','hh_{}'.format(y),'pop_{}'.format(y)]

    
    locals()['taz_out_jeru_{}'.format(y)]=taz_out_of_jeru_point.reset_index()[col].merge(taz_age,how='left',on='Taz_num')
    
   

חישוב לירושלים ואז הוספה של המרכיבים

In [987]:
#מייצר קטלוג לאזורים ללא התפלגות מדמוגרף

age_dis=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\התפלגות גילים\backround_files','age_des_types')

age_col_catalog=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

age_dis=age_dis.query('sector=="Arab" & age_cycle==2')[age_col_catalog]

In [988]:
taz['precent_ad']=0

In [989]:
x=2020

In [990]:
year=['2025','2030','2035','2040','2045','2050']

In [991]:
 for y in year:
        
    # לחבר למצב הנוכחי את הגידול והתפלגות שהוא צריך לקבל
    path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\עבודת_דמוגרף\SHP\pre_demo_growth_dislov_{}.shp'.format(y)

    demo=up_load_shp(path)

    taz_point=make_point(taz.reset_index()[['Taz_num','geometry']])

    demand=taz_point.sjoin(demo,how='left')
    taz_age=demand.reset_index()[col_age]
    
    demand=demand[['Taz_num','precent_ad']].set_index('Taz_num')


    
    taz['precent_ad']=demand['precent_ad']
    taz['precent_ad']=taz['precent_ad'].fillna(0)
    taz['precent_ad_{}'.format(y)]=taz['precent_ad']

    taz.loc[taz['precent_ad']>0,'pop_demand']=taz['current_pop']*taz['precent_ad']-taz['current_pop']
    taz.loc[taz['precent_ad']<=0,'pop_demand']=0
    taz['pop_demand_{}'.format(y)]=taz['pop_demand']
    taz['pop_kibolet_{}'.format(y)]=taz['pop_kibolet']

    # יצירת התפלגות,גידול וג"מ הממוצע של כל אחד מהערכים האלו בחומש

    # אזורים עם אוכלוסייה וללא התפלגות,גידול וג"מ יקבלו את הערך הממוצע הנל

    # מצב נוכחי גדל לפי אחוז

    # השוואה בין אוכלוסית ביקוש לקיבולת

    print("------------------------------------------")
    print(y)
    print("total_kibolet: ",taz['pop_kibolet'].sum().item())
    print("total_demand: ",taz['pop_demand'].sum().item())
    
    taz['pop_delta']=taz['pop_kibolet']-taz['pop_demand']
    taz['pop_delta_{}'.format(y)]=taz['pop_delta']

    # יש מקום לביקוש - לקחת את הביקוש
    taz.loc[taz['pop_delta']>0,'current_pop']=taz['pop_demand']+taz['current_pop']
    taz.loc[taz['pop_delta']>0,'pop_kibolet']=taz['pop_kibolet']-taz['pop_demand']
    taz.loc[taz['pop_delta']>0,'pop_{}_explanation'.format(y)]='pop_demand'


    # אין מקום לביקוש - לקחת את הקיבולת
    taz.loc[(taz['pop_delta']<=0)&(taz['pop_kibolet']>0),'current_pop']=taz['pop_kibolet']+taz['current_pop']
    taz.loc[(taz['pop_delta']<=0)&(taz['pop_kibolet']>0),'pop_kibolet']=0
    taz.loc[(taz['pop_delta']<=0)&(taz['pop_kibolet']>0),'pop_{}_explanation'.format(y)]='full_pop_kibolet'
    
    

    

    
    

    # לקבץ את כלל האוכלוסיה ללא בית לסל אחד

    pop_not_fit_in_taz=taz.loc[taz['pop_delta']<=0]['pop_delta'].sum().item()
    
    
    
    pop_not_fit_in_taz=abs(pop_not_fit_in_taz)
    

    
    print("pop_not_fit_in_taz: ",pop_not_fit_in_taz)

    # כימות כמה עוד מקום יש בקיבולת של ירושלים
    pop_kibolet_left=taz['pop_kibolet'].sum().item()
    
    print("pop_kibolet_left: ",pop_kibolet_left)
    
    

    


    if pop_not_fit_in_taz>pop_kibolet_left:

        print("pop_not_fit_in_taz>pop_kibolet_left")


        taz['current_pop']=taz['pop_kibolet']+taz['current_pop']
        taz['pop_kibolet']=0
        taz['pop_{}_explanation'.format(y)]=taz['pop_{}_explanation'.format(y)]+' +add_pop_to_fill_kibolet'
                
        pop_not_fit_in_jeru=pop_not_fit_in_taz-pop_kibolet_left
        
        print("pop_not_fit_in_jeru: ",pop_not_fit_in_jeru)
        
        taz_demo_pls['current_pop_pre']=taz_demo_pls['current_pop']/taz_demo_pls['current_pop'].sum().item()
        taz_demo_pls['add_pop_{}_from_jeru'.format(y)]=taz_demo_pls['current_pop_pre']*pop_not_fit_in_jeru
        taz_demo_pls['current_pop']=taz_demo_pls['current_pop']*taz_demo_pls['precent_add_pop_{}'.format(y)]
        taz_demo_pls['current_pop']=taz_demo_pls['current_pop']+taz_demo_pls['add_pop_{}_from_jeru'.format(y)]
        taz_demo_pls['pop_{}'.format(y)]=taz_demo_pls['current_pop']

    else:

        print("pop_not_fit_in_taz<=pop_kibolet_left")
        
        
        taz['add_pop_kibolet_from_pre']=taz['pop_kibolet']*(pop_not_fit_in_taz/pop_kibolet_left)
        taz['add_pop_kibolet_from_pre_{}'.format(y)]=taz['add_pop_kibolet_from_pre']
        taz['current_pop']=taz['add_pop_kibolet_from_pre']+taz['current_pop']
        taz['pop_kibolet']=taz['pop_kibolet']-taz['add_pop_kibolet_from_pre']
        taz.loc[taz['add_pop_kibolet_from_pre']>0,'pop_{}_explanation'.format(y)]=taz['pop_{}_explanation'.format(y)]+' +add_pop_by_pre_of_kibolet_pre'
        
        taz_demo_pls['current_pop']=taz_demo_pls['current_pop']*taz_demo_pls['precent_add_pop_{}'.format(y)]
        taz_demo_pls['pop_{}'.format(y)]=taz_demo_pls['current_pop']
        
    
    taz['pop_{}'.format(y)]=taz['current_pop']
    taz['add_pop_{}_{}'.format(y,str(x))]=taz['current_pop']-taz['pop_{}'.format(str(x))]
    taz.loc[(taz['add_pop_{}_{}'.format(y,str(x))]>0)&(taz['hh_size']<=0),'current_hh']=taz['current_hh']+taz['add_pop_{}_{}'.format(y,str(x))]/4.5
    
    taz.loc[(taz['add_pop_{}_{}'.format(y,str(x))]>0)&(taz['hh_size']>0),'current_hh']=taz['current_hh']+taz['add_pop_{}_{}'.format(y,str(x))]/4.5
    
    taz['hh_{}'.format(y)]=taz['current_hh']



    
    x=x+5
    col_current=['Taz_num','hh_{}'.format(y),'pop_{}'.format(y)]
    col=['pop_0',
     'pop_10',
     'pop_15',
     'pop_20',
     'pop_25',
     'pop_30',
     'pop_35',
     'pop_40',
     'pop_45',
     'pop_5',
     'pop_50',
     'pop_55',
     'pop_60',
     'pop_65',
     'pop_70',
     'pop_75up']
    
    df=taz.reset_index()[col_current].merge(taz_age,how='left',on='Taz_num')
    
    df=pd.concat([df,locals()['taz_out_jeru_{}'.format(y)]],axis=0)
    
    

    df1=df.loc[((df[col].sum(axis=1)==0)&(df['pop_{}'.format(y)]>0))]
    
    

    df1=df1[col_current]

    df1['join']=1

    age_dis['join']=1

    df1=df1.merge(age_dis,on='join',how='left').drop(columns='join')

    df=df.loc[~((df[col].sum(axis=1)==0)&(df['pop_{}'.format(y)]>0))]

    df=pd.concat([df,df1],axis=0)

    
    df=df.round(2)

    
    df[col]=df[col].div(df[col].sum(axis=1), axis=0)

    
    df[col] = df[col].apply(lambda x: x * df['pop_{}'.format(y)])
    
    df['pop_{}'.format(y)]=df[col].sum(axis=1)
           
    df=pd.concat([df,taz_demo_pls[['Taz_num','pop_{}'.format(y)]]],axis=0)  
   

    
    col=['Taz_num','hh_{}'.format(y),'pop_{}'.format(y),'pop_0',
         'pop_10',
         'pop_15',
         'pop_20',
         'pop_25',
         'pop_30',
         'pop_35',
         'pop_40',
         'pop_45',
         'pop_5',
         'pop_50',
         'pop_55',
         'pop_60',
         'pop_65',
         'pop_70',
         'pop_75up']
    
    df[col].to_excel(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\arab_pop\taz_Arab_Palestinian_{}.xlsx'.format(y),index=False)

------------------------------------------
2025
total_kibolet:  235869.44800000003
total_demand:  47430.234621365526
pop_not_fit_in_taz:  384.917684082295
pop_kibolet_left:  188824.13106271677
pop_not_fit_in_taz<=pop_kibolet_left
------------------------------------------
2030
total_kibolet:  188439.2133786345
total_demand:  51026.3190359476
pop_not_fit_in_taz:  2584.069729024657
pop_kibolet_left:  139996.96407171153
pop_not_fit_in_taz<=pop_kibolet_left
------------------------------------------
2035
total_kibolet:  137412.8943426869
total_demand:  51123.054362418145
pop_not_fit_in_taz:  13483.049203796638
pop_kibolet_left:  99772.88918406538
pop_not_fit_in_taz<=pop_kibolet_left
------------------------------------------
2040
total_kibolet:  86289.83998026873
total_demand:  49928.79164103979
pop_not_fit_in_taz:  24961.756650629217
pop_kibolet_left:  61322.804989858174
pop_not_fit_in_taz<=pop_kibolet_left
------------------------------------------
2045
total_kibolet:  36361.048339228946